<a href="https://colab.research.google.com/github/tluxxx/CandleStickStudies/blob/main/01_candlestick_studies_test_for_equal_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook that demonstrates test for equal means


In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from typing import Dict, Any

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from numba import njit

## 1. Generation of random series, testing for normality and plotting histograms

First we generate 3 series of random elements with the following features:


*  all series are non normal and independent   
*  series 1 and series 2: same distribution (exponential-distribution), but have different means
*  series 1 and series 3: different distributions (exponential and gamma-distribution), but have identical means

In [ ]:
# generation of series
np.random.seed(42)
series1 = np.random.exponential(scale=50, size=250)
series2 = np.random.exponential(scale=70, size=550)
series3 = np.random.gamma(shape=2, scale=25, size=350)

# Display basic statistics
print("=" * 90)
print("DESCRIPTIVE STATISTICS")
print("=" * 90)
stats_df = pd.DataFrame({
    'Series': ['Series 1', 'Series 2', 'Series 3'],
    'Mean': [series1.mean(), series2.mean(), series3.mean()],
    'Std Dev': [series1.std(), series2.std(), series3.std()],
    'Median': [np.median(series1), np.median(series2), np.median(series3)],
    'Skewness': [stats.skew(series1), stats.skew(series2), stats.skew(series3)],
    'Kurtosis': [stats.kurtosis(series1), stats.kurtosis(series2), stats.kurtosis(series3)],
    'Min': [series1.min(), series2.min(), series3.min()],
    'Max': [series1.max(), series2.max(), series3.max()],
    'Distribution': ['Exponential(50)', 'Exponential(70)', 'Gamma(2,25)']
})
stats_df


DESCRIPTIVE STATISTICS


,Series,Mean,Std Dev,Median,Skewness,Kurtosis,Min,Max,Distribution
0,Series 1,48.492896,46.365255,34.923874,1.302770,1.168239,0.253722,216.707317,Exponential(50)
1,Series 2,70.613570,71.217337,49.860237,2.014025,6.274788,0.762776,572.071192,Exponential(70)
2,Series 3,52.960806,35.494140,45.312085,1.125085,1.137002,2.182481,192.128649,"Gamma(2,25)"


In [ ]:
# plot of series

# preparations
series_list = [series1, series2, series3]
titles = [
    "Series 1 (Exp, scale=50)",
    "Series 2 (Exp, scale=70)",
    "Series 3 (Gamma, shape=2, scale=25)"
]
colors = ["red", "green", "blue"]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Box Plots", "Violin Plots")
)

# Left: Box plots
for series, title, color in zip(series_list, titles, colors):
    fig.add_trace(
        go.Box(
            y=series,
            name=title, marker_color=color,
            boxmean="sd"  # show mean and SD
        ),
        row=1, col=1,
    )

# Right: Violin plots with datapoints
for series, title, color in zip(series_list, titles, colors):
    fig.add_trace(
        go.Violin(
            y=series,
            name=title,
            box_visible=True,       # show inner box
            meanline_visible=True,  # show mean line
            points="all",           # show all datapoints
            jitter=0.5,             # spread datapoints
            scalemode="width",
            marker_color=color,
            line_color=color
        ),
        row=1, col=2,
    )


# Finetuning Layout

fig.update_layout(
    template="plotly_dark",
    title_text="Box Plots and Violin Plots with Data Points",
    showlegend=False,
    height=400, width=1000,
)

fig.update_yaxes(title_text="Value", row=1, col=1)
fig.update_yaxes(title_text="Value", row=1, col=2)

fig.show()

Now we test for Normality. We expect that for all distributions the Hypothesis "H0: Sample is normally distributed" is rejected.  

In [ ]:
print('*** NORMALITY TESTS (Shapiro-Wilk) ***')
print('H0: The data follows a normal distribution (α = 0.05) *** ')
print()
results = []
for series, name in [(series1, 'Series 1'), (series2, 'Series 2'), (series3, 'Series 3')]:
    stat, pval = stats.shapiro(series)
    results.append({
        'Series': name,
        'W-Statistic': round(stat, 4),
        'p-value': round(pval, 6),
        'Verdict': 'Fail to reject H₀' if pval > 0.05 else 'Reject H₀',
        'Decision': 'Normal' if pval > 0.05 else 'Non-Normal'
    })

df_normality = pd.DataFrame(results).set_index('Series')
df_normality

*** NORMALITY TESTS (Shapiro-Wilk) ***
H0: The data follows a normal distribution (α = 0.05) *** 



,W-Statistic,p-value,Verdict,Decision
Series,,,,
Series 1,0.8571,0.0,Reject H₀,Non-Normal
Series 2,0.8138,0.0,Reject H₀,Non-Normal
Series 3,0.9137,0.0,Reject H₀,Non-Normal


In [ ]:
# plotting
means = [np.mean(series1), np.mean(series2), np.mean(series3)]
series_list = [series1, series2, series3]

titles = [
    "Series 1 (Exp, scale=50)",
    "Series 2 (Exp, scale=70)",
    "Series 3 (Gamma, shape=2, scale=25)"
]

# preparations
colors = ["red", "green", "blue"]  # red, green, blue
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=titles
)

# Add traces for histograms
for i, series in enumerate(series_list, start=1):
    fig.add_trace(
        go.Histogram(
            x=series, nbinsx=30,
            histnorm="probability density",
            name=titles[i-1], marker_color=colors[i-1], opacity=0.75),
        row=1, col=i,
    )

# Add mean lines + annotations
for i, series in enumerate(series_list, start=1):
    mean_val = np.mean(series)
    hist_vals, _ = np.histogram(series, bins=30, density=True)
    ymax = hist_vals.max()  # max of density histogram

    # vertical dotted mean line
    fig.add_shape(
        type="line",
        x0=mean_val, x1=mean_val, y0=0, y1=ymax,
        line=dict(dash="dot", width=2, color="white"),
        row=1, col=i,
        )
    # annotation
    fig.add_annotation(
        x=mean_val, y=ymax,
        text=f"mean: {mean_val:.2f}", showarrow=True, arrowhead=2, arrowcolor="white",
        yshift=10, font=dict(color="white"),
        row=1, col=i,
        )

# finetuning layout
fig.update_layout(
    template="plotly_dark",
    title_text="Density Histograms with Mean Lines",
    showlegend=False,
    height=450, width=1200
)

# Shared axis titles
fig.update_xaxes(title_text="Value")
fig.update_yaxes(title_text="Density")

fig.show()



## 2. Step by Step trials of test for equal means

In [ ]:
# allocation of series, setting confidence-level
s1 = series1
s2 = series3
alpha =0.05
random_state=42

# observed differences
observed_diff = np.mean(s1) - np.mean(s2)       # T_obs: observed mean difference


### 2.1. Permutation Test

We test the equality of means:
- $H_0: \mu_1 = \mu_2$  
- $H_1: \mu_1 \neq \mu_2$  

We reject $H_0$ if $p < \alpha = 0.05$.

**Note:** The standard permutation test tests for equality of two distributions.
A special mean-centered permutation test is therefore required, when testing only for equality of means.

**Note:** the scipy.stats.permutation_test is made for standard permutation test (i.e. testing for equal distributions) and cannot be applied in our case directly.

The basis of the test is the observed difference between our two samples:
$T_{\text{obs}} = \bar{X}_1 - \bar{X}_2$.

We test how likely this observed difference is under the null hypothesis of equal means, by comparing it to a simulated distribution of differences drawn from samples that have been forced to share the same mean.

The test is executed in the following steps:

**Step 1** *(mean-centering — specific to the mean-centered permutation test)*:
Enforce $H_0$ by shifting both samples to a common pooled mean:
* Compute the pooled mean: $\bar{\mu} = \dfrac{\bar{X}_1 + \bar{X}_2}{2}$
* Shift each sample to $\bar{\mu}$, e.g. for $s_1$: $s^{*}_{1,i} = s_{1,i} - \bar{X}_1 + \bar{\mu}$

**Step 2** *(pooling)*:
Concatenate the two shifted samples into a single combined pool from which we will draw.

**Step 3** *(Monte Carlo resampling — repeated $B$ times)*:
* Randomly reshuffle the combined pool to obtain reshuffled sample $(b)$
* Split sample $(b)$ at $n_1$ to form two sub-samples of sizes $n_1$ and $n_2$
* Compute the mean difference for reshuffled sample $(b)$: $T^{(b)} = \bar{X}^{(b)}_1 - \bar{X}^{(b)}_2$
* Record $T^{(b)}$ in the history of simulated differences

**Step 4** *(decision)*:
* Calculate the p-value: the share of simulated $|T^{(b)}|$ that are **at least as large as** $|T_{\text{obs}}|$:
$$p = \frac{1}{B} \sum_{b=1}^{B} \mathbf{1}\left(|T^{(b)}| \geq |T_{\text{obs}}|\right)$$
* Compare $p$ to $\alpha$ and generate the verdict:
    * if $p < \alpha$ → **Reject $H_0$** (the observed difference is unlikely under $H_0$)
    * if $p \geq \alpha$ → **Fail to reject $H_0$** (insufficient evidence against $H_0$)





In [ ]:
%%time
# Code Version 1: not fully vectorized, but readable
n_resamples = 10_000

# --- Preparation ---
rng = np.random.default_rng(random_state)       # self-contained reproducible RNG

# --- Step 1: Mean-centering (enforce H0: mu1 = mu2) ---
pooled_mean = (np.mean(s1) + np.mean(s2)) / 2
s1_centered = s1 - np.mean(s1) + pooled_mean
s2_centered = s2 - np.mean(s2) + pooled_mean

# --- Step 2: Build the urn ---
combined = np.concatenate([s1_centered, s2_centered])
n1 = len(s1)

# --- Step 3: Monte Carlo resampling (B = n_resamples) ---
perm_diffs = []
for _ in range(n_resamples):
    rng.shuffle(combined)                        # random relabeling under H0
    perm_s1 = combined[:n1]
    perm_s2 = combined[n1:]
    perm_diffs.append(np.mean(perm_s1) - np.mean(perm_s2))   # record T^(b)

perm_diffs = np.array(perm_diffs)

# --- Step 4: Decision ---
perm_pval = np.sum(np.abs(perm_diffs) >= np.abs(observed_diff)) / n_resamples
perm_verdict = "Reject H0" if perm_pval < alpha else "Fail to reject H0"

print("Mean-Centered Permutation Test")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {observed_diff:+.4f}")
print(f"  B      (number of permutations)   : {n_resamples:,}")
print(f"  p-val  (share of |T^(b)| >= T_obs): {perm_pval:.4f}")
print(f"  Verdict                           : {perm_verdict}")


Mean-Centered Permutation Test
  H0: µ1 = µ2  |  confidence-level (alpha) = 0.05
  T_obs  (observed mean difference) : -4.4679
  B      (number of permutations)   : 10,000
  p-val  (share of |T^(b)| >= T_obs): 0.1816
  Verdict                           : Fail to reject H0
CPU times: user 300 ms, sys: 1.93 ms, total: 301 ms
Wall time: 378 ms


In [ ]:
%%time
# Version 2: fully vectorized
n_resamples = 10_000

# --- Preparation ---
rng = np.random.default_rng(random_state)       # self-contained reproducible RNG

# --- Step 1: Mean-centering (enforce H0: mu1 = mu2) ---
pooled_mean = (np.mean(s1) + np.mean(s2)) / 2
s1_centered = s1 - np.mean(s1) + pooled_mean
s2_centered = s2 - np.mean(s2) + pooled_mean

# --- Step 2: Build the urn ---
combined = np.concatenate([s1_centered, s2_centered])
n1      = len(s1)
n_total = len(combined)

# --- Select smallest integer dtype sufficient to index into combined ---
if n_total <= np.iinfo(np.uint8).max:        # n_total <= 255
    idx_dtype = np.uint8
elif n_total <= np.iinfo(np.uint16).max:     # n_total <= 65,535
    idx_dtype = np.uint16
elif n_total <= np.iinfo(np.uint32).max:     # n_total <= 4,294,967,295
    idx_dtype = np.uint32
else:
    idx_dtype = np.uint64

# --- Step 3: Vectorized Monte Carlo resampling (B = n_resamples) ---
# Generate all permutation indices at once: shape (B, n_total), cast to compact dtype
## rng.random((n_resamples, n_total)) generates a random number matrix (elements between ([0,1]) of shape (B, n_total)
## np.argsort(matrix, axis=1) returns the indicees that would sort the row from largest to smallest
## .astype() cast to defined dtype (to save memory)
## as a result we will have a differenc sequence of (n_max)-elements for each row (= simulation); altogether B rows in total
perm_indices = np.argsort(
    rng.random((n_resamples, n_total)), axis=1
).astype(idx_dtype)

# Apply all permutations simultaneously: shape (B, n_total)
perm_data = combined[perm_indices]

# Split and compute all T^(b) in one shot
perm_diffs = perm_data[:, :n1].mean(axis=1) - perm_data[:, n1:].mean(axis=1)

# --- Step 4: Decision ---
perm_pval   = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
perm_verdict = "Reject H0" if perm_pval < alpha else "Fail to reject H0"

print("Mean-Centered Permutation Test (vectorized)")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {observed_diff:+.4f}")
print(f"  B      (number of permutations)   : {n_resamples:,}")
print(f"  idx    (index dtype used)         : {idx_dtype.__name__}")
print(f"  p-val  (share of |T^(b)| >= T_obs): {perm_pval:.4f}")
print(f"  Verdict                           : {perm_verdict}")

Mean-Centered Permutation Test (vectorized)
  H0: µ1 = µ2  |  confidence-level (alpha) = 0.05
  T_obs  (observed mean difference) : -4.4679
  B      (number of permutations)   : 10,000
  idx    (index dtype used)         : uint16
  p-val  (share of |T^(b)| >= T_obs): 0.1891
  Verdict                           : Fail to reject H0
CPU times: user 217 ms, sys: 30.6 ms, total: 248 ms
Wall time: 370 ms


In [ ]:
%%time
# Version 3: not fully vectorized, but optimized (minimize summation), readable code
n_resamples = 10_000

# --- Preparation ---
rng = np.random.default_rng(random_state)

# --- Step 1: Mean-centering (enforce H0: mu1 = mu2) ---
pooled_mean = (np.mean(s1) + np.mean(s2)) / 2
s1_centered = s1 - np.mean(s1) + pooled_mean
s2_centered = s2 - np.mean(s2) + pooled_mean

# --- Step 2: Build the urn ---
combined = np.concatenate([s1_centered, s2_centered])
n1, n2 = len(s1), len(s2)
total_sum = combined.sum()

# --- Step 3: Monte Carlo resampling (B = n_resamples) ---
perm_diffs = np.empty(n_resamples)

for b in range(n_resamples):
    rng.shuffle(combined)                   # random relabelling centered series
    group1_sum = combined[:n1].sum()        # first sum
    group2_sum = total_sum - group1_sum     # avoiding to calculate the second sum
    perm_diffs[b] = (group1_sum / n1) - (group2_sum / n2) # array-operation rather than using list

perm_pval = np.mean(np.abs(perm_diffs) >= np.abs(observed_diff))
perm_verdict = "Reject H0" if perm_pval < alpha else "Fail to reject H0"

print("Mean-Centered Permutation Test (fast subset sampling)")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {observed_diff:+.4f}")
print(f"  B      (number of permutations)   : {n_resamples:,}")
print(f"  p-val  (share of |T^(b)| >= T_obs): {perm_pval:.4f}")
print(f"  Verdict                           : {perm_verdict}")

Mean-Centered Permutation Test (fast subset sampling)
  H0: µ1 = µ2  |  confidence-level (alpha) = 0.05
  T_obs  (observed mean difference) : -4.4679
  B      (number of permutations)   : 10,000
  p-val  (share of |T^(b)| >= T_obs): 0.1816
  Verdict                           : Fail to reject H0
CPU times: user 162 ms, sys: 1.96 ms, total: 164 ms
Wall time: 224 ms


In [ ]:
%%time
# version 4: usage of numba JIT-compilation

@njit
def _permutation_test_numba(a, b, n_perm, seed):
    """Mean-centered permutation test for continuous data (tests µ1 = µ2)."""
    na = len(a)
    nb = len(b)
    n  = na + nb

    # center both samples (impose H0: µ1 = µ2)
    mean_a = a.mean()
    mean_b = b.mean()

    combined = np.empty(n)
    for i in range(na):
        combined[i] = a[i] - mean_a
    for i in range(nb):
        combined[na + i] = b[i] - mean_b

    T_obs     = mean_a - mean_b
    abs_T_obs = T_obs if T_obs >= 0.0 else -T_obs  # abs(T_obs)

    rng_state = seed
    count = 0

    for _ in range(n_perm):
        for i in range(n - 1, 0, -1):
            rng_state = (rng_state * 6364136223846793005 + 1442695040888963407) & 0xFFFFFFFFFFFFFFFF
            j = rng_state % (i + 1)
            combined[i], combined[j] = combined[j], combined[i]

        s = 0.0
        for k in range(na):
            s += combined[k]
        diff     = s / na - (combined[na:].sum()) / nb
        abs_diff = diff if diff >= 0.0 else -diff

        if abs_diff >= abs_T_obs:
            count += 1

    return T_obs, count / n_perm

def permutation_test_centered_fast(a, b, n_perm=10_000, rng=None):
    """Drop-in replacement for permutation_test() — continuous cumrets."""
    seed = int(rng.integers(0, 2**31)) if rng is not None else 42
    return _permutation_test_numba(
        a.astype(np.float64),
        b.astype(np.float64),
        n_perm,
        seed
    )
# Permutation test using Numba-optimized version
n_resamples = 10_000
rng = np.random.default_rng(random_state)
observed_diff = np.mean(s1) - np.mean(s2)
T_obs, perm_pval = permutation_test_centered_fast(s1, s2, n_perm=n_resamples, rng=rng)
perm_verdict = "Reject H0" if perm_pval < alpha else "Fail to reject H0"

print("Mean-Centered Permutation Test (Numba-optimized)")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {T_obs:+.4f}")
print(f"  B      (number of permutations)   : {n_resamples:,}")
print(f"  p-val  (share of |T^(b)| >= T_obs): {perm_pval:.4f}")
print(f"  Verdict                           : {perm_verdict}")

Mean-Centered Permutation Test (Numba-optimized)
  H0: µ1 = µ2  |  confidence-level (alpha) = 0.05
  T_obs  (observed mean difference) : -4.4679
  B      (number of permutations)   : 10,000
  p-val  (share of |T^(b)| >= T_obs): 0.1836
  Verdict                           : Fail to reject H0
CPU times: user 1.18 s, sys: 58.1 ms, total: 1.23 s
Wall time: 1.4 s


In [ ]:
# --- Critical values of the permutation distribution ---
cv_low  = np.percentile(perm_diffs, 100 * alpha / 2)
cv_high = np.percentile(perm_diffs, 100 * (1 - alpha / 2))

# --- Plot ---
tobs_color = "crimson" if perm_verdict == "Reject H0" else "steelblue"
fig = go.Figure()

# Histogram of permutation distribution
fig.add_trace(go.Histogram(
    x=perm_diffs,
    nbinsx=80,
    name="T<sup>(b)</sup> distribution",
))

# Shaded rejection regions (both tails)
fig.add_vrect(x0=min(perm_diffs), x1=cv_low,
              fillcolor="red", opacity=0.20, layer="below", line_width=0,
              annotation_text="Rejection region", annotation_position="top left",
              annotation_font_size=11, annotation_font_color="white")
fig.add_vrect(x0=cv_high, x1=max(perm_diffs),
              fillcolor="red", opacity=0.20, layer="below", line_width=0,
              annotation_text="Rejection region", annotation_position="top right",
              annotation_font_size=11, annotation_font_color="white")

# Shaded acceptance region
fig.add_vrect(x0=cv_low, x1=cv_high,
              fillcolor="green", opacity=0.10, layer="below", line_width=0,
              annotation_text="Non-rejection region (95%)", annotation_position="bottom left",
              annotation_font_size=11, annotation_font_color="white")

# Critical value lines
for x, label in [(cv_low,  f"alpha/2 = {cv_low:.4f}"),
                 (cv_high, f"1-alpha/2 = {cv_high:.4f}")]:
    fig.add_vline(x=x, line_dash="dash", line_width=1.5, line_color="white")
    fig.add_annotation(
        x=x, y=0.5, yref="paper",
        text=label, showarrow=False,
        font_size=11, font_color="white", xanchor="left",
        bgcolor="rgba(0,0,0,0.3)",
    )

# T_obs vertical line
fig.add_vline(x=observed_diff, line_color=tobs_color, line_width=2.5)
fig.add_annotation(
    x=observed_diff, y=0.75, yref="paper",
    text=f"T_obs = {observed_diff:+.4f}<br>p = {perm_pval:.4f}<br>{perm_verdict}",
    showarrow=False,
    font_size=11, font_color=tobs_color, xanchor="left",
    bgcolor="rgba(0,0,0,0.3)",
)

fig.update_layout(
    title="Permutation Distribution of T(b) = X1(b) - X2(b)",
    xaxis_title="T(b) - simulated mean difference",
    yaxis_title="Count",
    template="plotly_dark",
    width=750, height=400,
)
fig.show()

## 2.2. Bootstrapping test

In [ ]:
%%time
# Version 1: not fully vectorized, but readable code

# --- Preparation ---
n_resamples = 10_000
rng = np.random.default_rng(random_state)

# --- Step 1: Resample with replacement (no H0 centering needed) ---
boot_diffs = np.empty(n_resamples)
for b in range(n_resamples):
    s1_star = rng.choice(s1, size=len(s1), replace=True)
    s2_star = rng.choice(s2, size=len(s2), replace=True)
    boot_diffs[b] = np.mean(s1_star) - np.mean(s2_star)     # record T^(b)

# --- Step 2: Basic (reverse percentile) CI ---
q_low  = np.quantile(boot_diffs, alpha / 2)
q_high = np.quantile(boot_diffs, 1 - alpha / 2)

ci_low  = 2 * observed_diff - q_high             # reversal around T_obs
ci_high = 2 * observed_diff - q_low              # reversal around T_obs

boot_verdict = "Reject H0" if not (ci_low <= 0 <= ci_high) else "Fail to reject H0"

print("Basic Bootstrap Confidence Interval")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {observed_diff:+.4f}")
print(f"  B      (number of resamples)      : {n_resamples:,}")
print(f"  95% CI (basic bootstrap)          : [{ci_low:.4f}, {ci_high:.4f}]")
print(f"  Verdict (0 outside CI => Reject)  : {boot_verdict}")

In [ ]:
%%time
# Version 2: scipy bootstrap CI
n_resamples  = 10_000
observed_diff = np.mean(s1) - np.mean(s2)  # fix the sign convention

def mean_diff(s1, s2, axis):
    return np.mean(s1, axis=axis) - np.mean(s2, axis=axis)

boot_result = stats.bootstrap(
    data=(s1, s2),
    statistic=mean_diff,
    n_resamples=n_resamples,
    confidence_level=1 - alpha,
    method='basic',
    alternative='two-sided',
    random_state=random_state
)

ci_low  = boot_result.confidence_interval.low
ci_high = boot_result.confidence_interval.high

# --- Sign correction: align CI with observed_diff ---
if np.sign(ci_low + ci_high) != np.sign(observed_diff):
    ci_low, ci_high = -ci_high, -ci_low

boot_verdict = "Reject H0" if not (ci_low <= 0 <= ci_high) else "Fail to reject H0"

print("Basic Bootstrap Confidence Interval (scipy)")
print(f"  H0: µ1 = µ2  |  confidence-level (alpha) = {alpha}")
print(f"  T_obs  (observed mean difference) : {observed_diff:+.4f}")
print(f"  B      (number of resamples)      : {n_resamples:,}")
print(f"  95% CI (basic bootstrap)          : [{ci_low:.4f}, {ci_high:.4f}]")
print(f"  Verdict (0 outside CI => Reject)  : {boot_verdict}")

In [ ]:
# visualisation
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Sample Distributions",
                                    "Bootstrap Distribution of T^(b)",
                                    "Confidence Intervals vs H0",
                                    "ECDF of T^(b)"))

# ── Panel 1: Box plots ───────────────────────────────────────────────────────
fig.add_trace(go.Box(y=s1, name="s1"), row=1, col=1)
fig.add_trace(go.Box(y=s2, name="s2"), row=1, col=1)

# ── Panel 2: Histogram of boot_diffs ────────────────────────────────────────
fig.add_trace(go.Histogram(x=boot_diffs, name="T^(b)", nbinsx=80), row=1, col=2)
fig.add_vline(x=observed_diff, line=dict(color="black", dash="dot"), row=1, col=2)
fig.add_vline(x=ci_low,        line=dict(color="red",   dash="dash"), row=1, col=2)
fig.add_vline(x=ci_high,       line=dict(color="red",   dash="dash"), row=1, col=2)

# ── Panel 3: CI comparison ───────────────────────────────────────────────────
ci_basic_low  = 2*observed_diff - np.quantile(boot_diffs, 1 - alpha/2)
ci_basic_high = 2*observed_diff - np.quantile(boot_diffs, alpha/2)

for i, (lo, hi, label) in enumerate([
    (ci_basic_low, ci_basic_high, "Basic"),
    (ci_low,       ci_high,       "BCa")
]):
    fig.add_trace(go.Scatter(x=[lo, hi], y=[i, i], mode="lines+markers",
                             name=label), row=2, col=1)

fig.add_vline(x=0,             line=dict(color="green", dash="dash"), row=2, col=1)
fig.add_vline(x=observed_diff, line=dict(color="black", dash="dot"),  row=2, col=1)
fig.update_yaxes(tickvals=[0, 1], ticktext=["Basic", "BCa"], row=2, col=1)

# ── Panel 4: ECDF ────────────────────────────────────────────────────────────
sorted_diffs = np.sort(boot_diffs)
ecdf_y = np.arange(1, len(sorted_diffs)+1) / len(sorted_diffs)
fig.add_trace(go.Scatter(x=sorted_diffs, y=ecdf_y, mode="lines",
                         name="ECDF"), row=2, col=2)
fig.add_vline(x=ci_low,        line=dict(color="red",   dash="dash"), row=2, col=2)
fig.add_vline(x=ci_high,       line=dict(color="red",   dash="dash"), row=2, col=2)
fig.add_vline(x=observed_diff, line=dict(color="black", dash="dot"),  row=2, col=2)

# ── Layout ───────────────────────────────────────────────────────────────────
fig.update_layout(title_text=f"Bootstrap CI | T_obs={observed_diff:+.4f} | " f"95% CI (BCa): [{ci_low:.4f}, {ci_high:.4f}] | "f"{boot_verdict}",
                  template='plotly_dark',
                  height=400,width=900,
                  )
fig.show()

## 2.3 Welch-test

The **Hypotheses**: We test: $H₀: µ₁ = µ₂ $  against $ H₁: µ₁ ≠ µ₂ $ (two-tailed)

The **Test Statistic** (Welch's t-statistic) is:
$$t = \frac{\bar{x}_1 − \bar{x}_2} {\sqrt{\frac{s^2_1}{n_1} + \frac{S^2_2}{n_2}}}$$ where $\bar{x}_i$ are sample means, $s_i^2$​ are sample variances (ddof=1), and $n_i$ are sample sizes. Under H₀ this statistic follows a t-distribution — but with how many degrees of freedom?

**Welch-Satterthwaite Degrees of Freedom:**

Unlike Student's t-test, Welch does not assume equal variances, so the degrees of freedom are approximated:

$$ df = \frac{\left(\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}\right)^2}{\frac{(s_1^2/n_1)^2}{n_1-1} + \frac{(s_2^2/n_2)^2}{n_2-1}}$$

This is generally non-integer, which is why you see df=31.7 or similar in the plot.

The Decision Rule
We reject H₀ if $|t| > t_{\text{crit}}$​, where $ t_{\text{crit}} = t_{1-\alpha/2, df} $, df​ is the quantile of the t-distribution. Equivalently, we reject if $p < \alpha$.

The p-value is the probability of observing a t-statistic at least as extreme as ours under H₀: $p = 2 \cdot P(T > |t|) \quad \text{where } T \sim t_{df}$

In [ ]:
# WELCH'S t-test

# 1. Sample statistics (inputs to the test statistic) ---
n1, n2 = len(s1), len(s2)                        # sample sizes
m1, m2 = np.mean(s1), np.mean(s2)                # sample means  -> µ1, µ2
v1, v2 = np.var(s1, ddof=1), np.var(s2, ddof=1)  # sample variances s1², s2² (unbiased, ddof=1)

# 2. Test statistic  t = (m1 - m2) / sqrt(v1/n1 + v2/n2)
t_stat, p_val = stats.ttest_ind(s1, s2, equal_var=False)  # scipy computes t and p in one call

# 3. Welch-Satterthwaite degrees of freedom  df = (v1/n1 + v2/n2)² / [(v1/n1)²/(n1-1) + (v2/n2)²/(n2-1)]
df = (v1/n1 + v2/n2)**2 / ((v1/n1)**2/(n1-1) + (v2/n2)**2/(n2-1))

# 4. Critical value and p-value
#       t_crit = t_{1-alpha/2, df}  -> the boundary of the rejection region
#       p = 2 * P(T > |t|)    -> two-tailed, computed by scipy above
t_crit = stats.t.ppf(1 - alpha/2, df=df)

# 5. Decision
verdict = "Reject H0" if p_val < alpha else "Fail to reject H0"

# 6. Summary
print("Welch's t-test")
print(f"  n1={n1}, n2={n2}")
print(f"  mean difference   :   {m1-m2:.4f}")
print(f"  pooled SE         :   {np.sqrt(v1/n1 + v2/n2):.4f}")
print(f"  df (Satterthwaite):   {df:.2f}")
print(f"  t-statistic       :   {t_stat:.4f}")
print(f"  t-critical (±)    :   {t_crit:.4f}")
print(f"  p-value           :   {p_val:.4f}")
print(f"  alpha             :   {alpha}")
print(f"  Verdict           :   {verdict}")


In [ ]:
# PLOT
x = np.linspace(-4, 4, 500)
y = stats.t.pdf(x, df=df)          # t-distribution under H0 with our df

fig = make_subplots(rows=1, cols=2, subplot_titles=("Distributions", f"t-distribution (df={df:.1f})"))

# --- Left: histograms --- Visual check of the two empirical distributions
for sample, name in zip([s1, s2], ["s1", "s2"]):
    fig.add_trace(go.Histogram(x=sample, name=name, opacity=0.5, histnorm="probability density"), row=1, col=1)
fig.update_layout(barmode="overlay")

# --- Right: t-distribution with rejection regions: |t| > t_crit, area = alpha/2 on each side
for mask, label in [
    (x <= -t_crit, "Rejection region (α/2 each side)"),
    (x >=  t_crit, None)
]:
    fig.add_trace(go.Scatter(
        x=x[mask], y=y[mask], fill="tozeroy",
        fillcolor="rgba(255,0,0,0.3)", line=dict(color="rgba(0,0,0,0)"),
        name=label, showlegend=label is not None
    ), row=1, col=2)

# The t-distribution ( theoretical null distribution of t under H0)
fig.add_trace(go.Scatter(x=x, y=y, line=dict(color="black"), name="t-distribution"), row=1, col=2)

# Observed t-statistic — where our data lands on the null distribution
fig.add_vline(x=t_stat, line=dict(color="blue", dash="dash"),
              annotation_text=f"t={t_stat:.3f}", row=1, col=2)

# Critical values — boundary between reject / fail to reject
fig.add_vline(x=-t_crit, line=dict(color="red", dash="dot"),
              annotation_text=f"-t_crit={-t_crit:.3f}", row=1, col=2)
fig.add_vline(x= t_crit, line=dict(color="red", dash="dot"),
              annotation_text=f"t_crit={t_crit:.3f}", row=1, col=2)

fig.update_layout(
    title=f"Welch's t-test | t={t_stat:.3f} | p={p_val:.4f} | <b>{verdict}</b>",
    height=350, width = 1000
)

fig.show()

# 3. Test of all 3 Series

In [ ]:
tests = {
    "series1 vs series2": (series1, series2),
    "series1 vs series3": (series1, series3),
    "series2 vs series3": (series2, series3),
}

##3.1 Welch-t tests

In [ ]:
def welch_test(
        a: np.ndarray,
        b: np.ndarray,
        comparison: str = "",
        alpha: float = 0.05,
        ) -> dict:
    '''Welch's t-test for equality of means (two-tailed).
    Args:
        a          : first sample
        b          : second sample
        comparison : label for the comparison (default "")
        alpha      : significance level (default 0.05)
    Returns:
        dict with comparison, mean_a, mean_b, mean_diff, t_stat, t_crit, df, p_value, verdict, performance
    '''
    result = stats.ttest_ind(a, b, equal_var=False)
    t_crit = stats.t.ppf(1 - alpha/2, df=result.df)
    verdict = "Reject H0" if result.pvalue < alpha else "Fail to reject H0"

    if verdict == "Fail to reject H0":
        performance = "not detectable"
    elif np.mean(a) > np.mean(b):
        performance = "outperform"
    else:
        performance = "underperform"

    return {
        "comparison"        : comparison,
        "welch_mean_a"      : np.mean(a),
        "welch_mean_b"      : np.mean(b),
        "welch_mean_diff"   : np.mean(a) - np.mean(b),
        "welch_t_stat"      : result.statistic,
        "welch_t_crit"      : t_crit,
        "welch_df"          : result.df,
        "welch_p_value"     : result.pvalue,
        "welch_verdict"     : verdict,
        "welch_performance" : performance,
    }

In [ ]:
# --- Loop ---
results = []
for name, (a, b) in tests.items():
    results.append(welch_test(a, b, comparison=name, alpha=alpha))

df_welch = pd.DataFrame(results).round(4)
df_welch

## 3.2. permutation test

In [ ]:
def permutation_test(
        a: np.ndarray,
        b: np.ndarray,
        comparison: str = "",
        n_resamples: int = 10_000,
        alpha: float = 0.05,
        random_state: int = None,
        ) -> dict:
    '''Mean-centered permutation test for equality of means (two-tailed).
    Args:
        a, b        : first/second sample
        comparison  : label for the comparison (default "")
        n_resamples : number of Monte Carlo permutations (default 10_000)
        alpha       : significance level (default 0.05)
        random_state: seed for reproducibility (default None)
    Returns:
        dict with comparison, mean_a, mean_b, mean_diff, n_resamples, p_value, verdict, performance
    '''
    # preparations
    rng = np.random.default_rng(random_state)

    # Step 1: observed difference
    observed_diff = np.mean(a) - np.mean(b)

    # Step 2: mean-centering (enforce H0: mu1 = mu2) and building urn
    pooled_mean = (np.mean(a) + np.mean(b)) / 2
    combined = np.concatenate([a - np.mean(a) + pooled_mean,
                                b - np.mean(b) + pooled_mean])
    n_a = len(a)

    # Step 3: Monte Carlo resampling
    perm_diffs = []
    for _ in range(n_resamples):
        rng.shuffle(combined)
        perm_diffs.append(np.mean(combined[:n_a]) - np.mean(combined[n_a:]))

    # Step 4: p-value, verdict and performance
    perm_pval = np.sum(np.abs(perm_diffs) >= np.abs(observed_diff)) / n_resamples
    verdict = "Reject H0" if perm_pval < alpha else "Fail to reject H0"

    if verdict == "Fail to reject H0":
        performance = "not detectable"
    elif observed_diff > 0:
        performance = "outperform"
    else:
        performance = "underperform"

    return {
        "comparison"       : comparison,
        "perm_mean_a"      : np.mean(a),
        "perm_mean_b"      : np.mean(b),
        "perm_mean_diff"   : observed_diff,
        "n_resamples"      : n_resamples,
        "perm_p_value"     : perm_pval,
        "perm_verdict"     : verdict,
        "perm_performance" : performance,
    }

In [ ]:
# execution of permutation test
results = []
for name, (a, b) in tests.items():
    row = permutation_test(a, b, comparison=name, n_resamples=10_000, alpha=alpha, random_state=random_state)
    results.append(row)

df_perm = pd.DataFrame(results).round(4)
df_perm



In [ ]:
# generating one dataframe for both tests

perm_results, welch_results = [], []
for name, (a, b) in tests.items():
    perm_results.append(permutation_test(a, b, comparison=name, n_resamples=10_000, alpha=alpha, random_state=random_state))
    welch_results.append(welch_test(a, b, comparison=name, alpha=alpha))

df_perm  = pd.DataFrame(perm_results).round(4)
df_welch = pd.DataFrame(welch_results).round(4)

df_combined = pd.merge(df_perm, df_welch, on="comparison")
df_combined[['comparison', 'perm_verdict','perm_performance', 'welch_verdict', 'welch_performance']]


###RETHINKING


In [ ]:
# ==========================
# Clean Bootstrap CI plot
# ==========================
fig2 = go.Figure()

comparisons_list = ['Series 1 vs Series 2',
                    'Series 1 vs Series 3',
                    'Series 2 vs Series 3']
y_positions = [3, 2, 1]
colors_ci = ['purple', 'orange', 'brown']

for i, row in enumerate(results):
    ci_low, ci_high = row['bootstrap']['confidence_interval']
    mean_diff = row['observed_mean_difference']

    # Decision text
    decision_text = "Reject H0" if row['bootstrap']['reject_H0'] else "Fail to reject H0"

    # Add mean + error bar (CI)
    fig2.add_trace(go.Scatter(
        x=[mean_diff],
        y=[y_positions[i]],
        mode='markers',
        name=comparisons_list[i],
        marker=dict(size=12, color=colors_ci[i]),
        error_x=dict(
            type='data',
            symmetric=False,
            array=[ci_high - mean_diff],
            arrayminus=[mean_diff - ci_low],
            thickness=3,
            width=20,
            color=colors_ci[i]
        )
    ))

    # Add text annotation for decision
    fig2.add_annotation(
        x=ci_high + (ci_high - ci_low)*0.05,  # slightly to the right of CI
        y=y_positions[i],
        text=decision_text,
        showarrow=False,
        font=dict(color="white", size=12)
    )

# H0 reference line
fig2.add_vline(
    x=0,
    line_dash="dash",
    line_color="white",
    line_width=2,
    annotation_text="H0: diff=0",
    annotation_position="top"
)

# Layout
fig2.update_layout(
    template="plotly_dark",
    title="Bootstrap 95% Confidence Intervals for Mean Differences",
    xaxis_title="Mean Difference (Bootstrap)",
    yaxis=dict(tickmode='array', tickvals=y_positions, ticktext=comparisons_list),
    height=500,
    width=1000,
    showlegend=True
)

fig2.show()




Experiments for Code Clarification: